postgresql에서 label_math_ele뽑아옴

In [1]:
import pandas as pd
from pymongo import MongoClient
from sqlalchemy import create_engine

db_config = {
        'host': "10.41.2.78",
        'port': "5432",
        'user': "postgres",
        'pw': "qwe123",
        'db': "project",
        'table_name': "label_math_ele"
    }
    
db_url = f"postgresql://{db_config['user']}:{db_config['pw']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"
engine = create_engine(db_url)

query = f"SELECT * FROM {db_config['table_name']}"
label_math_ele = pd.read_sql(query, engine)

In [2]:
label_math_ele

,from_id,from_semester,from_chapter_name,to_id,to_chapter_name,to_semester,from_g,to_g
0,116,중등-중7-1학기,수와연산 > 소인수분해 > 소인수분해,7944,곱셈 > 곱셈식을 알아볼까요,초등-초2-1학기,중등,초등
1,2615,초등-초5-1학기,자연수의 혼합 계산 > 덧셈과 뺄셈이 섞여 있는 식을 계산해 볼까요,7933,덧셈과 뺄셈 > 세 수의 계산을 해 볼까요,초등-초2-1학기,초등,초등
2,2616,초등-초5-1학기,자연수의 혼합 계산 > 곱셈과 나눗셈이 섞여 있는 식을 계산해 볼까요,2068,곱셈과 나눗셈 > 세 자릿수에 두 자릿수를 곱해 볼까요,초등-초4-1학기,초등,초등
3,2616,초등-초5-1학기,자연수의 혼합 계산 > 곱셈과 나눗셈이 섞여 있는 식을 계산해 볼까요,2073,곱셈과 나눗셈 > 세 자릿수를 두 자릿수로 나누어 볼까요 (1),초등-초4-1학기,초등,초등
4,2617,초등-초5-1학기,"자연수의 혼합 계산 > 덧셈, 뺄셈, 곱셈이 섞여 있는 식을 계산해 볼까요",2615,자연수의 혼합 계산 > 덧셈과 뺄셈이 섞여 있는 식을 계산해 볼까요,초등-초5-1학기,초등,초등
...,...,...,...,...,...,...,...,...
1605,11269,초등-초4-1학기,"큰 수 > 십만, 백만, 천만을 알아볼까요",7268,큰 수 > 다섯 자릿수를 알아볼까요,초등-초4-1학기,초등,초등
1606,11270,초등-초4-1학기,큰 수 > 억과 조를 알아볼까요,11269,"큰 수 > 십만, 백만, 천만을 알아볼까요",초등-초4-1학기,초등,초등
1607,11271,초등-초4-1학기,큰 수 > 억과 조를 알아볼까요,11270,큰 수 > 억과 조를 알아볼까요,초등-초4-1학기,초등,초등
1608,8481,초등-초4-1학기,각도 > 각도의 합과 차는 얼마일까요,7310,덧셈과 뺄셈 > 덧셈을 해 볼까요 (3),초등-초3-1학기,초등,초등


mongoDB로 부터 JSON파일을 DataFrame으로 변환

In [3]:
import pandas as pd
from pymongo import MongoClient

# MongoDB 연결 설정
username = 'root'
password = 'qwe123'
host = '127.0.0.1'
port = 27017
client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}/')
db = client['project']

# 컬렉션 선택
qa_collection = db['1_correct_answer']
irt_collection = db['2_questionIRT']
# taker_irt_collection = db['3_candidateIRT']

In [4]:
# 필요한 필드만 선택하여 데이터를 가져오는 함수
def fetch_collection_as_dataframe(collection, fields):
    data = list(collection.find({}, fields))
    return pd.DataFrame(data)

# 필요한 필드만 선택
qa_fields = {'testID': 1, 'assessmentItemID': 1, 'learnerID': 1, 'answerCode': 1, '_id': 0}
irt_fields = {'testID': 1, 'assessmentItemID': 1, 'knowledgeTag': 1, '_id': 0}

# 데이터프레임으로 변환
qa_df = fetch_collection_as_dataframe(qa_collection, qa_fields)
irt_df = fetch_collection_as_dataframe(irt_collection, irt_fields)

# 병합 전 필요한 컬럼만 유지 및 인덱스 설정
qa_df = qa_df.set_index(['testID', 'assessmentItemID'])
irt_df = irt_df.set_index(['testID', 'assessmentItemID'])

# 최적화된 병합
merged_df = pd.merge(qa_df, irt_df, left_index=True, right_index=True, how='left').reset_index()

# 병합 후 필요한 컬럼만 선택
merged_df = merged_df[['learnerID', 'answerCode', 'knowledgeTag']]
merged_df['knowledgeTag'] = pd.to_numeric(merged_df['knowledgeTag'], errors='coerce', downcast='integer')


In [5]:
merged_df

,learnerID,answerCode,knowledgeTag
0,A010000003,1,5844
1,A010000006,1,5844
2,A010000008,1,5844
3,A010000016,1,5844
4,A010000019,1,5844
...,...,...,...
1677353,A090000446,0,2648
1677354,A090000468,0,2648
1677355,A090000483,1,2648
1677356,A090000485,1,2648


postgresql에서 education_2022 dataframe으로 변환 + 전처리

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# 연결 정보 설정
db_config = {
    'host': "127.0.0.1",
    'port': "5432",
    'user': "postgres",
    'pw': "qwe123",
    'db': "project",
    'location': "localhost_target",
    'engine': "postgre",
    'table_name': "education_2022"
}

# 데이터베이스 URL 구성
db_url = f"postgresql://{db_config['user']}:{db_config['pw']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"

# SQLAlchemy를 사용하여 PostgreSQL에 연결하는 엔진 생성
engine = create_engine(db_url)

# SQL 쿼리 정의 (테이블의 모든 데이터를 선택)
query = f"SELECT * FROM {db_config['table_name']}"

# 쿼리 실행 및 결과를 DataFrame으로 변환
education_2022 = pd.read_sql(query, engine)

# 'Unnamed: 6' 컬럼을 제거
education_2022 = education_2022.drop(columns=['Unnamed: 6'], errors='ignore')

# '계열화' 컬럼의 NaN 값을 '정보 없음'으로 대체
education_2022['계열화'] = education_2022['계열화'].fillna('정보 없음')


In [7]:
education_2022


,ID,Name,영역명,내용요소,계열화,grade
0,2048,큰 수 > 억과 조를 알아볼까요 / 큰 수 > 억과 조를 알아볼까요,수와 연산,다섯 자리 이상의 수,A-4-7,4-1
1,2049,큰 수 > 뛰어 세기를 해 볼까요,수와 연산,다섯 자리 이상의 수,A-4-9,4-1
2,2050,큰 수 > 수의 크기를 비교해 볼까요 / 큰 수 > 수의 크기를 비교해 볼까요,수와 연산,다섯 자리 이상의 수,A-4-10,4-1
3,2052,각도 > 어느 각이 더 클까요,도형과 측정(도형),도형의 기초,C-3-3,4-1
4,2053,각도 > 각의 크기는 얼마일까요,도형과 측정(측정),각도( ° ),D-8-1,4-1
...,...,...,...,...,...,...
637,10741,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1
638,5141,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1
639,5142,제곱근과 실수 > 제곱근과 실수 > 실수의 대소 관계,수와 연산,제곱근과 실수,정보 없음,9-1
640,5921,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1


학생이 맞춘 knowledgeTag와 틀린 knowledgeTag의 빈도수가 많은걸 가져와 이후, 이전 학습을 추천해줌

In [8]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np


# NaN 값을 빈 문자열로 대체
label_math_ele['from_chapter_name'].fillna('', inplace=True)
label_math_ele['to_chapter_name'].fillna('', inplace=True)


def extract_semester(semester_str):
    if pd.isna(semester_str):
        return ''
    match = re.search(r'(\d+)-(\d+)', semester_str)
    return (int(match.group(1)), int(match.group(2))) if match else (None, None)


def parse_series_order(order_str):
    if not order_str:  # None 또는 빈 문자열일 때
        return ('ZZZ', float('inf'), float('inf'))  # Default value (정렬 우선순위 가장 낮음)
    match = re.match(r'([A-Z]+)-(\d+)-(\d+)', order_str)
    if match:
        return (match.group(1), int(match.group(2)), int(match.group(3)))
    return ('ZZZ', float('inf'), float('inf'))  # Default value for malformed input (정렬 우선순위 가장 낮음)


def get_unique_id_name_pairs(df):
    to_id_name = dict(zip(df['to_id'], df['to_chapter_name']))
    from_id_name = dict(zip(df['from_id'], df['from_chapter_name']))
    all_ids = set(to_id_name.keys()).union(set(from_id_name.keys()))
    unique_id_name_pairs = {}
    unique_id_semesters = {}
    for id_ in all_ids:
        to_name = to_id_name.get(id_, '')
        from_name = from_id_name.get(id_, '')
        if to_name and from_name:
            unique_id_name_pairs[id_] = f'{to_name} / {from_name}'
        elif to_name:
            unique_id_name_pairs[id_] = to_name
        elif from_name:
            unique_id_name_pairs[id_] = from_name
        from_semester = df[df['from_id'] == id_]['from_semester'].values
        to_semester = df[df['to_id'] == id_]['to_semester'].values
        if len(from_semester) > 0:
            unique_id_semesters[id_] = extract_semester(from_semester[0])
        elif len(to_semester) > 0:
            unique_id_semesters[id_] = extract_semester(to_semester[0])
        else:
            unique_id_semesters[id_] = (None, None)
    return unique_id_name_pairs, unique_id_semesters


# 본개념과 ID-이름 쌍, 학기 정보 가져오기
unique_id_name_pairs, unique_id_semesters = get_unique_id_name_pairs(label_math_ele)


# education_2022 데이터를 ID를 기준으로 매칭할 수 있도록 딕셔너리 생성
education_mapping = education_2022.set_index('ID').to_dict('index')
predecessors = defaultdict(list)
successors = defaultdict(list)
to_chapter_names = dict(zip(label_math_ele['to_id'], label_math_ele['to_chapter_name']))
from_chapter_names = dict(zip(label_math_ele['from_id'], label_math_ele['from_chapter_name']))
for _, row in label_math_ele.iterrows():
    from_id = row['from_id']
    to_id = row['to_id']
    predecessors[from_id].append(to_id)
    successors[to_id].append(from_id)


# 특정 학습자에 대한 맞은 문제와 틀린 문제 집계
def analyze_student_performance(learner_id, df):
    student_df = df[df['learnerID'] == learner_id].copy()
    if student_df.empty:
        print(f"학습자 {learner_id}의 데이터가 없습니다.")
        return None, None
    # 'answerCode'의 데이터 타입 확인 및 변환
    if student_df['answerCode'].dtype == 'object':
        student_df.loc[:, 'answerCode'] = pd.to_numeric(student_df['answerCode'], errors='coerce')
    # 맞은 문제와 틀린 문제 분류
    correct_df = student_df[student_df['answerCode'] == 1]
    incorrect_df = student_df[student_df['answerCode'] == 0]
    # 각 문제의 knowledgeTag 수집
    correct_knowledge_tags = correct_df['knowledgeTag'].dropna()
    incorrect_knowledge_tags = incorrect_df['knowledgeTag'].dropna()
    return correct_knowledge_tags.tolist(), incorrect_knowledge_tags.tolist()


# 빈도수가 가장 높은 태그를 찾는 함수
def get_most_common_tag(tags_list):
    all_tags = [tag for sublist in tags_list for tag in sublist]  # 중첩 리스트를 평탄화
    if all_tags:  # 리스트가 비어있지 않은 경우
        return pd.Series(all_tags).value_counts().idxmax()
    return None


# 특정 개념에 대한 본개념, 선수개념, 후속개념을 출력하는 함수
def get_concepts(node_id):
    main_concept = {
        '본개념': node_id,
        '본개념_Chapter_Name': unique_id_name_pairs.get(node_id, '정보 없음'),
        '학년-학기': unique_id_semesters.get(node_id, (None, None))
    }
    # 선수학습과 후속학습을 '계열화' 기준으로 정렬
    main_concept['선수학습'] = sorted(
        predecessors.get(node_id, []),
        key=lambda x: parse_series_order(education_mapping.get(x, {}).get('계열화', ''))
    , reverse=True)
    main_concept['후속학습'] = sorted(
        successors.get(node_id, []),
        key=lambda x: parse_series_order(education_mapping.get(x, {}).get('계열화', ''))
    )
    return main_concept


def add_education_info(df, id_col):
    df['영역명'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('영역명', '정보 없음'))
    df['내용요소'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('내용요소', '정보 없음'))
    df['계열화'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('계열화', '정보 없음'))
    return df


# NaN 값을 특정 문자열로 대체
def replace_nan_with_string(df, replace_str='정보 없음'):
    return df.replace({np.nan: replace_str})

def get_concepts_df(node_id):
    # Get concept data
    concepts = get_concepts(node_id)
    
    # Create main concept DataFrame
    main_concept_df = pd.DataFrame([{
        '본개념_ID': concepts['본개념'],
        '본개념_Chapter_Name': concepts['본개념_Chapter_Name'],
        '학년-학기': f"{concepts['학년-학기'][0]}-{concepts['학년-학기'][1]}"
    }])
    
    # Add additional education info
    main_concept_df = add_education_info(main_concept_df, '본개념_ID')
    
    # Replace NaN with specific string
    main_concept_df = replace_nan_with_string(main_concept_df)
    print("Main Concept DataFrame:", main_concept_df)  # For debugging
    
    # Create predecessors DataFrame
    predecessors_df = pd.DataFrame({
        '선수학습_ID': concepts['선수학습'],
        '선수학습_Chapter_Name': [unique_id_name_pairs.get(id, '정보 없음') for id in concepts['선수학습']],
        '학년-학기': [f"{unique_id_semesters.get(id, (None, None))[0]}-{unique_id_semesters.get(id, (None, None))[1]}" for id in concepts['선수학습']]
    })
    
    # Add additional education info
    predecessors_df = add_education_info(predecessors_df, '선수학습_ID')
    
    # Replace NaN with specific string
    predecessors_df = replace_nan_with_string(predecessors_df)
    print("Predecessors DataFrame:", predecessors_df)  # For debugging
    
    # Create successors DataFrame
    successors_df = pd.DataFrame({
        '후속학습_ID': concepts['후속학습'],
        '후속학습_Chapter_Name': [unique_id_name_pairs.get(id, '정보 없음') for id in concepts['후속학습']],
        '학년-학기': [f"{unique_id_semesters.get(id, (None, None))[0]}-{unique_id_semesters.get(id, (None, None))[1]}" for id in concepts['후속학습']]
    })
    
    # Add additional education info
    successors_df = add_education_info(successors_df, '후속학습_ID')
    
    # Replace NaN with specific string
    successors_df = replace_nan_with_string(successors_df)
    print("Successors DataFrame:", successors_df)  # For debugging
    
    return main_concept_df, predecessors_df, successors_df


# 사용자로부터 학습자 ID 입력 받기
learner_id_input = input("학습자 ID를 입력하세요: ").strip()
# 학습자 ID에 대한 데이터 분석
correct_knowledge_tags, incorrect_knowledge_tags = analyze_student_performance(learner_id_input, merged_df)
if correct_knowledge_tags is not None and incorrect_knowledge_tags is not None:
    # 결과에서 가장 빈도수 높은 태그 추출
    most_common_correct_tag = get_most_common_tag([correct_knowledge_tags])
    most_common_incorrect_tag = get_most_common_tag([incorrect_knowledge_tags])
    print(f"\n학습자 {learner_id_input}의 맞은 문제에서 가장 빈도수가 높은 knowledgeTag: {most_common_correct_tag}")
    print("맞은 문제 관련 본개념, 후속개념 정보:")
    correct_main_concept_df, _, correct_successors_df = get_concepts_df(most_common_correct_tag)  # 선수학습을 무시하기 위해 '_' 사용
    # 틀린 문제의 knowledgeTag로 개념 정보 출력
    print(f"\n학습자 {learner_id_input}의 틀린 문제에서 가장 빈도수가 높은 knowledgeTag: {most_common_incorrect_tag}")
    print("틀린 문제 관련 본개념, 선수개념 정보:")
    incorrect_main_concept_df, incorrect_predecessors_df, _ = get_concepts_df(most_common_incorrect_tag)  # 후속학습을 무시하기 위해 '_' 사용


학습자 A010000008의 맞은 문제에서 가장 빈도수가 높은 knowledgeTag: 7597
맞은 문제 관련 본개념, 후속개념 정보:
Main Concept DataFrame:    본개념_ID                                 본개념_Chapter_Name 학년-학기    영역명  \
0    7597  덧셈과 뺄셈 > 덧셈을 해 볼까요 (2) / 덧셈과 뺄셈 > 덧셈을 해 볼까요 (2)   1-1  수와 연산   

                내용요소    계열화  
0  두 자리 수 범위의 덧셈과 뺄셈  A-2-4  
Predecessors DataFrame:    선수학습_ID                                  선수학습_Chapter_Name 학년-학기    영역명  \
0     7595  덧셈과 뺄셈 > 더하기는 어떻게 나타낼까요 / 덧셈과 뺄셈 > 더하기는 어떻게 나타낼까요   1-1  수와 연산   
1     6409  덧셈과 뺄셈 > 모으기와 가르기를 해 볼까요 (1) / 덧셈과 뺄셈 > 모으기와 가...   1-1  수와 연산   

                내용요소    계열화  
0  두 자리 수 범위의 덧셈과 뺄셈  A-2-2  
1  두 자리 수 범위의 덧셈과 뺄셈  A-2-1  
Successors DataFrame:     후속학습_ID                                  후속학습_Chapter_Name 학년-학기    영역명  \
0      7620          50까지의 수 > 십몇을 알아볼까요 / 50까지의 수 > 십몇을 알아볼까요   1-1  수와 연산   
1      7621  50까지의 수 > 10개씩 묶어 세어 볼까요 / 50까지의 수 > 10개씩 묶어 세...   1-1  수와 연산   
2      6555  50까지의 수 > 50까지의 수를 세어 볼까요 / 50까지의 수 > 50까지의 수를...   1-1  수와 연산   
